In [ ]:
import os, glob
import numpy as np
import pandas as pd
import tensorflow as tf
from PIL import Image
import matplotlib.pyplot as plt

from keras.models import *
from keras.layers import *
# from keras.layers import Input, merge, Conv2D, MaxPooling2D, UpSampling2D, Dropout, Cropping2D, concatenate, Activation,Conv2DTranspose, add
from keras.layers import BatchNormalization
from keras.optimizers import *
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, History, ReduceLROnPlateau
from keras.callbacks import EarlyStopping
from keras.utils.generic_utils import get_custom_objects

from sklearn.metrics import roc_curve, auc, precision_recall_curve, average_precision_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix

from keras.utils import multi_gpu_model

from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
# from keras.utils.training_utils import multi_gpu_model

os.environ["CUDA_VISIBLE_DEVICES"] = "6,7"


In [ ]:
epsilon = K.epsilon()
gamma = 5
alpha = 0.6
beta = 0.6

def recall(y_target, y_pred):
    # clip(t, clip_value_min, clip_value_max) : clip_value_min~clip_value_max 이외 가장자리를 깎아 낸다
    y_target_yn = K.round(K.clip(y_target, 0, 1))
    y_pred_yn = K.round(K.clip(y_pred, 0, 1)) 
    count_true_positive = K.sum(y_target_yn * y_pred_yn)
    count_true_positive_false_negative = K.sum(y_target_yn)

    recall = count_true_positive / (count_true_positive_false_negative + K.epsilon())
    return recall

def precision(y_target, y_pred):
    y_pred_yn = K.round(K.clip(y_pred, 0, 1))
    y_target_yn = K.round(K.clip(y_target, 0, 1))
    count_true_positive = K.sum(y_target_yn * y_pred_yn) 
    count_true_positive_false_positive = K.sum(y_pred_yn)

    precision = count_true_positive / (count_true_positive_false_positive + K.epsilon())
    return precision

def dice_coef(y_true, y_pred, smooth=0.001):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

def iou_coef(y_true, y_pred, smooth=1):
    intersection = K.sum(K.abs(y_true * y_pred), axis=[1,2,3])
    union = K.sum(y_true,[1,2,3])+K.sum(y_pred,[1,2,3])-intersection
    iou = K.mean((intersection + smooth) / (union + smooth), axis=0)
    return iou

def dice_coef_loss(y_true, y_pred):
    return 1 - dice_coef(y_true, y_pred)


get_custom_objects().update({
    
    'precision' : precision,
    'recall' : recall,
    'dice_coef' : dice_coef,
    'iou_coef' : iou_coef,
    'dice_coef_loss' : dice_coef_loss,
        
})

img_size = 512
depth = 4
filter_sn = 32

In [ ]:
inputs = Input((img_size, img_size,1), dtype='float32')

def encoder_unet(inputs, filters, kernel_size= 3):
    conv = Conv2D(filters, kernel_size, activation=None, padding='same')(inputs)
    conv = BatchNormalization()(conv)
    conv = Activation('relu')(conv)
    conv = Conv2D(filters, kernel_size, activation=None, padding='same')(conv)
    conv = BatchNormalization()(conv)
    conv = Activation('relu')(conv)
#     pool = MaxPooling2D(pool_size=(2, 2))(conv)
    
    return conv

def decoder_unet(inputs, filters, kernel_size= 3, concate_num=depth-1):
    up = concatenate([Conv2DTranspose(filters, (2, 2), strides=(2, 2), padding='same')(inputs), encoders[concate_num]], axis=3)
    conv = Conv2D(filters, kernel_size, activation=None, padding='same')(up)
    conv = BatchNormalization()(conv)
    conv = Activation('relu')(conv)
    conv = Conv2D(filters, kernel_size, activation=None, padding='same')(conv)
    conv = BatchNormalization()(conv)
    conv = Activation('relu')(conv)
    
    return conv

def unet(depth=4, filter_sn=32, img_size=512):
    global encoders
    encoders=[]
    
    inputs = Input((img_size, img_size,1), dtype='float32')
    for down in range(0,depth+1):
#         print(down)
        if down == 0:
            encoder_conv = encoder_unet(inputs, filters=filter_sn, kernel_size= 3)
            encoders.append(encoder_conv)
            encoder_conv = MaxPooling2D(pool_size=(2, 2))(encoder_conv)
#             print(filter_sn)
        elif down>0 and down<depth:
            encoder_conv = encoder_unet(encoder_conv, filters=filter_sn*(2**down), kernel_size= 3)
            encoders.append(encoder_conv)
            encoder_conv = MaxPooling2D(pool_size=(2, 2))(encoder_conv)
#             print(filter_sn*(2**down))
        else:
            encoder_conv = encoder_unet(encoder_conv, filters=filter_sn*(2**(down-1)), kernel_size= 3)
#             encoders.append(encoder_conv)
#             print(filter_sn*(2**(down-1)))
#             print(encoder_conv.shape)
    

#         print(filter_sn*(2**depth))
        
    for up in range(0,depth):
        if up == 0:
            decoder_conv= decoder_unet(encoder_conv, filters=filter_sn*(2**(down-1)), kernel_size= 3, concate_num=len(encoders)-1)
        else:
            decoder_conv= decoder_unet(decoder_conv, filters=filter_sn*(2**(depth-up-1)),kernel_size= 3, concate_num=depth-up-1)
#         print('decoders:', decoder_conv.shape)
    last_conv = Conv2D(1, (1, 1), activation='sigmoid')(decoder_conv)

    model = Model(inputs=inputs, outputs=last_conv) 
    
    return model

In [ ]:
model=unet(depth=4, filter_sn=32, img_size=512)
model = multi_gpu_model(model, gpus=2)

model.summary()

In [ ]:
def calculate_performance(pred, test_y):
    tp=0
    fp=0
    tn=0
    fn=0
    alpha = 0.0001
    for cm in range(len(test_y)):
        if list(test_y)[cm]==0:
            if pred[cm]<0.5:
                tn+=1
            else:
                fn+=1
        else:
            if pred[cm]<0.5:
                fp+=1
            else:
                tp+=1
    print(tp, fp, tn, fn)
    sensitivity= (tp+alpha)/(tp+fn+alpha)
    specificity= (tn+alpha)/(tn+fp+alpha) 
    acc = (tp+tn+alpha)/(tp+fp+tn+fn+alpha)
    prec = (tp+alpha)/(tp+fp+alpha)

    return tp, fp, tn, fn, sensitivity, specificity, acc, prec

In [ ]:
def cal_dsc(y_true, y_pred, smooth=0.001 ):
    dice = np.sum(y_true[y_pred==1.0])*2.0 / (np.sum(y_true) + np.sum(y_pred)+smooth)
    return dice

def confusion_matrix_jw_2(y_true, y_pred,TP=0, FP=0, TN=0,FN=0, smooth=0.001 ):
    ravel_yt = y_true.ravel()
    ravel_yp = y_pred.ravel()
    for ry in range(len(ravel_yt)):
        if ravel_yt[ry]==1 and ravel_yp[ry]==1:
            TP+=1
        elif ravel_yt[ry]==0 and  ravel_yp[ry]==0:
            TN+=1
        elif ravel_yt[ry]==0 and  ravel_yp[ry]==1:
            FP+=1
        else:
            FN+=1 
    return TP, TN, FP, FN

def performance(y_true, y_pred, smooth=0.001):
    print(y_true.shape, y_pred.shape)
    
#     TP, TN, FP, FN = confusion_matrix(y_true, y_pred).ravel()
    TP, TN, FP, FN = confusion_matrix_jw_2(y_true, y_pred)
    accuracy = (TP + TN)/(TP+TN+FP+FN+smooth)
    sensitivity = TP/(TP+FN+smooth)
    specificity = TN/(TN+FP+smooth)
    precision = TP/(TP+FP+smooth)
    return accuracy, sensitivity, specificity, precision

def binary_result(gt, results):
    
    Bresult = np.ndarray(results.shape, dtype=np.bool)
    Gtruth = np.ndarray(gt.shape, dtype=np.bool)
    # for ri in range(len(all_results)):
    #     all_results[ri][all_results[ri]>=0.5] = 1.0
    #     all_results[ri][all_results[ri]<0.5] = 0
    Bresult[results>=0.5]= 1.0
    Bresult[results<0.5]= 0
    
    Gtruth[gt>=0.5]= 1.0
    Gtruth[gt<0.5]= 0
    return Gtruth, Bresult

In [ ]:
train_x = np.load('./new_train_img_323.npy')
train_y = np.load('./new_train_label_323.npy')
test_x =np.load('./new_test_img_323.npy')
test_y = np.load('./new_test_label_323.npy')

train_x = train_x/255
train_y = train_y/255
test_x = test_x/255
test_y = test_y/255

print("loading data done") 
print(train_x.shape)
print(train_y.shape)
print(test_x.shape)
print(test_y.shape)

In [ ]:
num_val_samples = len(train_x) // k
print(num_val_samples)

In [ ]:
k=5
savepath='./210518_Unet_001'
model_type = 'unet_001'
date='0518'
os.makedirs(savepath+'/npy', exist_ok=True)
os.makedirs(savepath+'/result', exist_ok=True)
os.makedirs(savepath+'/model', exist_ok=True)
num_val_samples = len(train_x) // k
for i in range(5):
    print('처리중인 폴드 #', i)
    val_x = train_x[i * num_val_samples: (i + 1) * num_val_samples]  # 검증 데이터 준비: k번째 분할
    val_y = train_y[i * num_val_samples: (i + 1) * num_val_samples]

    partial_train_x = np.concatenate(  # 훈련 데이터 준비: 다른 분할 전체
        [train_x[:i * num_val_samples],
         train_x[(i + 1) * num_val_samples:]],
        axis=0)
    partial_train_y = np.concatenate(
        [train_y[:i * num_val_samples],
         train_y[(i + 1) * num_val_samples:]],
        axis=0)
    
    print(i * num_val_samples, (i + 1) * num_val_samples)
    np.save(savepath+'/npy/{}_train_x_k{}'.format(model_type, str(i)), partial_train_x)
    np.save(savepath+'/npy/{}_train_y_k{}'.format(model_type,str(i)), partial_train_y) 
    
    input_img = Input(shape=(512,512,1))
    model = unet(depth=4, filter_sn=32, img_size=512)
#     model.summary()

    data='SciRep323'
    model = multi_gpu_model(model, gpus=2)
    option='kfold{}_batch16'.format(str(i))
    batch_size = 16
    monitor = 'val_loss'

    model.compile(optimizer=Adam(lr=0.001), loss=dice_coef_loss, metrics=['accuracy', recall, precision, dice_coef])
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=10, verbose=1, min_delta=1e-4)
    earlystopper = EarlyStopping(monitor='val_loss',patience=30, verbose=1)
    model_checkpoint = ModelCheckpoint(savepath+'/model/{}_{}_{}_{}.h5'.format(data,model_type,option,date), monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=False)

    callbacks_list = [reduce_lr, model_checkpoint, earlystopper]
    
    results = model.fit(partial_train_x, partial_train_y, batch_size=16, epochs=500, verbose=1,validation_data=(val_x,val_y), shuffle=False, callbacks=callbacks_list)
    

In [ ]:
# i=0
perform_index = 5
performances = np.ndarray((k+2,perform_index), dtype=np.float32)
indexList = np.expand_dims(np.asarray(['dice', 'accuracy', 'sensitivity', 'specificity', 'precision']), axis=0)
rows = np.expand_dims(np.asarray([model_type,'fold0','fold1','fold2','fold3','fold4','Mean','STD']).T
                      , axis=1)
# for a in range(perform_index):
#     performances[0][a] = performanceList[a]
# test_x= np.load(savepath+'/npy/test_x_fold{}.npy'.format(str(i)))
# test_y= np.load(savepath+'/npy/test_y_fold{}.npy'.format(str(i)))
for i in range(k):
    print(i)
    option='kfold{}_batch16'.format(str(i))
    test_model = load_model(savepath+'/model/{}_{}_{}_{}.h5'.format(data,model_type,option,date))
    
    _loss, _acc, _precision, _recall,_dice_coef = test_model.evaluate(test_x, test_y, batch_size=8, verbose=1)
    print('loss: {:.3f}, accuracy: {:.3f}, precision: {:.3f}, recall: {:.3f}, dice_coef:{:.3f}'.format(_loss, _acc*100, _precision*100, _recall*100, _dice_coef*100))
    
    test_result= test_model.predict(test_x, batch_size=64, verbose=1)
    np.save(savepath+'/npy/test_result_fold{}.npy'.format(str(i)), test_result)
    
    Bresult, Gresult = binary_result(test_result, test_y)
    dice = cal_dsc(Gresult, Bresult)
    accuracy, sensitivity, specificity, precision = performance(Gresult, Bresult)
    performanceList =[dice, accuracy, sensitivity, specificity, precision]
    print(performanceList)
    for a in range(perform_index):
        performances[i][a] = performanceList[a]
performances[k] = np.mean(performances[:k], axis=0)
performances[k+1] = np.std(performances[:k], axis=0)
print(performances.shape, indexList.shape)
alls = np.concatenate((indexList, performances), axis=0)
np.save(savepath+'/npy/{}_performanceList_meanNstd.npy'.format(model_type), performances)
perform = np.concatenate((rows, alls), axis=1)
all_perform = pd.DataFrame(perform)
all_perform.to_csv(savepath+'/result/all_{}_perform.csv'.format(model_type))
print(all_perform)

In [ ]:
indexList = np.expand_dims(np.asarray(['dice', 'accuracy', 'sensitivity', 'specificity', 'precision']), axis=0)

performances[k] = np.mean(performances[:k], axis=0)
performances[k+1] = np.std(performances[:k], axis=0)
print(performances.shape, indexList.shape)
alls = np.concatenate((indexList, performances), axis=0)
np.save(savepath+'/npy/{}_performanceList_meanNstd.npy'.format(model_type), performances)
perform = np.concatenate((rows, alls), axis=1)
all_perform = pd.DataFrame(perform)
all_perform.to_csv(savepath+'/result/all_{}_perform.csv'.format(model_type))
print(all_perform)

In [ ]:
import pandas as pd
performances_r = performances[1:6]
performanceList = ['dice', 'accuracy', 'sensitivity', 'specificity', 'precision']

stds = np.std(performances_r, axis=0)
means = np.mean(performances_r, axis=0)
expd_stds = np.expand_dims(stds, axis=0)
expd_means = np.expand_dims(means, axis=0)
confidences = np.concatenate((expd_means, expd_stds), axis=0)
values = np.concatenate((performances_r, confidences), axis=0)
rows = np.expand_dims(np.asarray([model_type,'fold0','fold1','fold2','fold3','fold4','Mean','STD']).T
                      , axis=1)

arr_perf = np.expand_dims(np.asarray(performanceList), axis=0)

alls = np.concatenate((arr_perf, values), axis=0)
print(rows.shape, alls.shape)
perform = np.concatenate((rows, alls), axis=1)
print(perform)

all_perform = pd.DataFrame(perform)
all_perform.to_csv(savepath+'/result/all_{}_performs_only.csv'.format(model_type))

# performances[k+1] = np.std(performances, axis=0)
# performances[k+] = np.mean(performances, axis=0)

# perform = np.concatenate((np.asarray(performancesList), performances), axis=0)
np.save(savepath+'/npy/{}_performanceList_meanNstd.npy'.format(model_type), values)

In [ ]:
performances[k+2] = np.std(performances, axis=0)
performances[k+3] = np.mean(performances, axis=0)
perform = np.concatenate((np.asarray(performancesList), performances), axis=0)
np.save(savepath+'/npy/{}_performanceList.npy'.format(model_type), performances)
all_perform = pd.DataFrame(performances)
all_perform.to_csv(savepath+'/result/all_{}_perform.csv'.format(model_type))

In [ ]:
print(accuracy, sensitivity, specificity, precision)

In [ ]:
print(accuracy, sensitivity, specificity, precision)

In [ ]:
print(accuracy, sensitivity, specificity, precision)

In [ ]:
print(Bresult.shape, Gresult.shape)
plt.figure(figsize=(10,5))
plt.subplot(1,2,1)
plt.imshow(Bresult[0][:,:,0], cmap='gray')
plt.subplot(1,2,2)
plt.imshow(Gresult[0][:,:,0], cmap='gray')

In [ ]:
print(performanceList)

In [ ]:
print("loading data")

train_image = np.load('../../data/segmentation/data300/train_img300.npy')
train_label = np.load('../../data/segmentation/data300/train_label300.npy')
test_image =np.load('../../data/segmentation/data300/test_image_512.npy')
test_label = np.load('../../data/segmentation/data300/test_label_512.npy')

train_image = train_image/255
train_label = train_label/255
test_image = test_image/255
test_label = test_label/255

print("loading data done")
print(train_image.shape)
print(train_label.shape)
print(test_image.shape)
print(test_label.shape)

In [ ]:
results = model.predict(test_image, batch_size=8, verbose=1)
np.save('./result/prob_npy/{}_{}.npy'.format(model_name,date), results)

In [ ]:
def preds_Thresh(preds_test,Thresh_value):
    
    preds_mask = np.ndarray((len(test_label),img_size, img_size, 1), dtype=np.float32)
  
    preds_mask0 = preds_test[:,:,:,0] > Thresh_value
  
    preds_mask[:,:,:,0] = preds_mask0*1
    
    return preds_mask

In [ ]:
preds_mask = preds_Thresh(results,0.5)
check_len = 5
for toto in range(check_len):
    plt.figure(figsize=(15, 15))
    plt.subplot(2,3,1)
    plt.imshow(test_image[toto,:,:,0],cmap='gray')
    plt.axis('off')
    plt.subplot(2,3,2)
    plt.imshow(test_label[toto,:,:,0],cmap='gray')
    plt.axis('off')
    plt.subplot(2,3,3)
    plt.imshow(test_image[toto,:,:,0],cmap='gray')
    plt.imshow(test_label[toto,:,:,0],cmap='Reds' ,alpha=0.6)
    plt.axis('off')
    plt.subplot(2,3,4)
    plt.imshow(test_image[toto,:,:,0],cmap='gray')
    plt.axis('off')
    plt.subplot(2,3,5)
    plt.imshow(preds_mask[toto,:,:,0],cmap='gray')
    plt.axis('off')
    plt.subplot(2,3,6)
    plt.imshow(test_image[toto,:,:,0],cmap='gray')
    plt.imshow(preds_mask[toto,:,:,0],cmap='Reds' ,alpha=0.6)
    plt.axis('off')